In [1]:
!git clone https://github.com/razyrick/Mask-RCNN_TF2.14.0
%cd Mask-RCNN_TF2.14.0

Cloning into 'Mask-RCNN_TF2.14.0'...
remote: Enumerating objects: 230, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 230 (delta 25), reused 15 (delta 15), pack-reused 192
Receiving objects: 100% (230/230), 74.89 MiB | 40.60 MiB/s, done.
Resolving deltas: 100% (73/73), done.
/kaggle/working/Mask-RCNN_TF2.14.0


In [2]:
!pip install -r requirements.txt
!python3 -m pip install tensorflow[and-cuda]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.8/793.8 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 77.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 84

In [3]:
!pip install setuptools==58.2.0
!python3 setup.py install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 946.1/946.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.0.3
    Uninstalling setuptools-69.0.3:
      Successfully uninstalled setuptools-69.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
arviz 0.18.0 requires setuptools>=60.0.0, but you have setuptools 58.2.0 which is incompatible.
conda 24.5.0 requires packaging>=23.0, but you have packaging 21.3 which is incompatible.
conda 24.5.0 requires setuptools>=60.0.0, but you have setuptools 58.2.0 which is incompatible.
pyldavis 3.4.1 requires numpy>=1.24.2, but you have numpy 1.23.5 which is incompatible.
tensorflow-decision-forests 1.8.1 requires tensorflow~=2.15.0, but you have tensorflow 2.14.0 which is incompatible.
tensorfl

In [4]:
from mrcnn.config import Config

class CustomConfig(Config):
    NAME = "custom"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 2
    NUM_CLASSES = 1 + 1  # Background + your single class
    STEPS_PER_EPOCH = 479
    LEARNING_RATE = 0.00001
    DETECTION_MIN_CONFIDENCE = 0.9

In [5]:
import os
import json
import numpy as np
import skimage.io
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
import skimage.draw

class CustomDataset(Dataset):
    def load_custom(self, dataset_dir, subset):
        """Load a subset of the custom dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add class
        self.add_class("custom", 1, "coconut")

        # Path to annotations
        annotations_path = os.path.join(dataset_dir, f"annotations_{subset}.json")

        # Load annotations
        with open(annotations_path) as f:
            annotations = json.load(f)

        # Add images
        for image_info in annotations['images']:
            image_id = image_info['id']
            image_path = os.path.join(dataset_dir, image_info['file_name'])
            image_annotations = [ann for ann in annotations['annotations'] if ann['image_id'] == image_id]
            self.add_image(
                "custom",
                image_id=image_id,
                path=image_path,
                width=image_info['width'],
                height=image_info['height'],
                annotations=image_annotations
            )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a custom dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "custom":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        annotations = info['annotations']
        masks = []
        class_ids = []

        for annotation in annotations:
            class_id = annotation['category_id']
            mask = np.zeros((info['height'], info['width']), dtype=np.uint8)
            for seg in annotation['segmentation']:
                # Ensure coordinates are within image dimensions
                seg = np.array(seg).reshape(-1, 2)
                seg[:, 0] = np.clip(seg[:, 0], 0, info['width'] - 1)
                seg[:, 1] = np.clip(seg[:, 1], 0, info['height'] - 1)
                rr, cc = skimage.draw.polygon(seg[:, 1], seg[:, 0])
                mask[rr, cc] = 1
            masks.append(mask)
            class_ids.append(class_id)

        if not masks:
            print(f"No masks found for image_id: {image_id}")

        masks = np.stack(masks, axis=-1) if masks else np.empty((info['height'], info['width'], 0), dtype=np.uint8)
        class_ids = np.array(class_ids, dtype=np.int32)
        return masks, class_ids

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "custom":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

2024-07-05 15:27:15.583395: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 15:27:15.583459: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 15:27:15.583502: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
#prepare train
from mrcnn.model import MaskRCNN

config = CustomConfig()
model = MaskRCNN(mode="training", config=config, model_dir="/kaggle/working/logs")

coco_weights_path = "/kaggle/input/cocooo/mask_rcnn_coco.h5"  # Update this path to the actual location of the weights file
model.load_weights(coco_weights_path, by_name=True, exclude=[
    "mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

dataset_train = CustomDataset()
dataset_train.load_custom("/kaggle/input/coconut/dataset/dataset/train", "train")
dataset_train.prepare()

dataset_val = CustomDataset()
dataset_val.load_custom("/kaggle/input/coconut/dataset/dataset/val", "val")
dataset_val.prepare()

Viewing Logs:
Once downloaded, you can use TensorBoard on your local machine to visualize the logs:

Open a terminal.

Navigate to the directory containing the downloaded logs directory.

Run TensorBoard:

tensorboard --logdir=./logs

Open your web browser and go to http://localhost:6006 to view TensorBoard and explore the loss and validation loss graphs.

In [7]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime
callbacks = [
    TensorBoard(log_dir='logs/{}'.format(datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))),
    EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001, restore_best_weights=True, verbose=1)  # Adjust parameters as needed
]
model.train(
    dataset_train, dataset_val,
    learning_rate=config.LEARNING_RATE,
    epochs=1000,
    layers='heads',
    custom_callbacks=callbacks
)




Starting at epoch 0. LR=1e-05

Checkpoint Path: /kaggle/working/logs/custom20240705T1527/mask_rcnn_custom_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
mrcnn_mask_bn4         (TimeDistributed)
mrcnn_bbox_fc          (TimeDistributed)
mrcnn_mask_deconv      (Tim

/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/1000
479/479 [==============================] - ETA: 0s - batch: 239.0000 - size: 2.0000 - loss: 1.5257 - rpn_class_loss: 0.0518 - rpn_bbox_loss: 0.4324 - mrcnn_class_loss: 0.1896 - mrcnn_bbox_loss: 0.6471 - mrcnn_mask_loss: 0.2048

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


479/479 [==============================] - 993s 2s/step - batch: 239.0000 - size: 2.0000 - loss: 1.5257 - rpn_class_loss: 0.0518 - rpn_bbox_loss: 0.4324 - mrcnn_class_loss: 0.1896 - mrcnn_bbox_loss: 0.6471 - mrcnn_mask_loss: 0.2048 - val_loss: 1.2676 - val_rpn_class_loss: 0.0544 - val_rpn_bbox_loss: 0.4329 - val_mrcnn_class_loss: 0.1552 - val_mrcnn_bbox_loss: 0.4465 - val_mrcnn_mask_loss: 0.1784
Epoch 2/1000
479/479 [==============================] - 969s 2s/step - batch: 239.0000 - size: 2.0000 - loss: 1.0992 - rpn_class_loss: 0.0495 - rpn_bbox_loss: 0.4239 - mrcnn_class_loss: 0.1139 - mrcnn_bbox_loss: 0.3369 - mrcnn_mask_loss: 0.1749 - val_loss: 0.9112 - val_rpn_class_loss: 0.0246 - val_rpn_bbox_loss: 0.3142 - val_mrcnn_class_loss: 0.1019 - val_mrcnn_bbox_loss: 0.3055 - val_mrcnn_mask_loss: 0.1649
Epoch 3/1000
479/479 [==============================] - 929s 2s/step - batch: 239.0000 - size: 2.0000 - loss: 0.8967 - rpn_class_loss: 0.0310 - rpn_bbox_loss: 0.3585 - mrcnn_class_loss: 0.0